In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math
from datetime import timedelta

from Clust.clust.ML.regression_JS import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.data import df_set_data
from Clust.clust.data import data_interface

from Clust.setting import influx_setting_KETI as ins
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)

from clust.preprocessing.dataPreprocessing import DataPreprocessing
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ML.tool import scaler as ml_scaler

# Set Param

In [2]:
feature_list = ["out_blacksphere_temp", "out_coai", "out_h2s", 'out_humi', 'out_nh3', 'out_pm10', 'out_temp', 'out_ultraviolet_rays']
target_col = "out_pm10"

get_y_data_transformParameter ={
    "future_step": 1,# 0~x
    "past_step":24,# 1~y
    "feature_col": feature_list,
    "target_col":target_col 
}   

param = {
    "x_features" : 8, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

bk_name = "air_outdoor_kweather"
ms_name = "OC3CL200010"

shuffle = False
model_file_path = "./ada_rnn_model/test_adarnn_model_by_origin_data"

# 22.01.01 ~ 22.01.11 데이터 없음
test_start_time = "2022-01-01 00:00:00"
test_end_time = "2022-04-29 23:59:00"

# get Data

## Data Read

In [3]:
origin_data = db_client.get_data_by_time(test_start_time, test_end_time, bk_name, ms_name)

In [4]:
origin_data

,out_blacksphere_temp,out_co,out_coai,out_coci,out_h2s,out_humi,out_light_intensity,out_nh3,out_no2,out_noise,out_o3,out_pm10,out_pm25,out_so2,out_temp,out_ultraviolet_rays
time,,,,,,,,,,,,,,,,
2022-01-11 15:34:00+00:00,-4.6,-9999.0,79.0,75.0,0.0,33.0,-9999.0,0.0,-9999.0,55.0,-9999.0,101.0,14.0,-9999.0,-2.1,1.3
2022-01-11 15:36:00+00:00,-4.6,-9999.0,93.0,89.0,0.0,33.0,-9999.0,0.0,-9999.0,53.0,-9999.0,23.0,11.0,-9999.0,-2.1,1.3
2022-01-11 15:38:00+00:00,-4.7,-9999.0,93.0,89.0,0.0,33.0,-9999.0,0.0,-9999.0,50.0,-9999.0,23.0,11.0,-9999.0,-2.3,1.3
2022-01-11 15:40:00+00:00,-4.7,-9999.0,93.0,89.0,0.0,33.0,-9999.0,0.0,-9999.0,53.0,-9999.0,23.0,10.0,-9999.0,-2.4,1.3
2022-01-11 15:42:00+00:00,-4.8,-9999.0,93.0,89.0,0.0,33.0,-9999.0,0.0,-9999.0,51.0,-9999.0,21.0,10.0,-9999.0,-2.4,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29 23:50:00+00:00,10.8,-9999.0,100.0,100.0,0.0,82.0,-9999.0,0.0,-9999.0,42.0,-9999.0,0.0,0.0,-9999.0,11.3,1.3
2022-04-29 23:52:00+00:00,10.8,-9999.0,100.0,100.0,0.0,82.0,-9999.0,0.0,-9999.0,43.0,-9999.0,0.0,0.0,-9999.0,11.3,1.3
2022-04-29 23:54:00+00:00,11.0,-9999.0,100.0,100.0,0.0,84.0,-9999.0,0.0,-9999.0,44.0,-9999.0,0.0,0.0,-9999.0,11.3,1.3


In [5]:
origin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39102 entries, 2022-01-11 15:34:00+00:00 to 2022-04-29 23:58:00+00:00
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  39102 non-null  float64
 1   out_co                39102 non-null  float64
 2   out_coai              39102 non-null  float64
 3   out_coci              39102 non-null  float64
 4   out_h2s               39102 non-null  float64
 5   out_humi              39102 non-null  float64
 6   out_light_intensity   39102 non-null  float64
 7   out_nh3               39102 non-null  float64
 8   out_no2               39102 non-null  float64
 9   out_noise             39102 non-null  float64
 10  out_o3                39102 non-null  float64
 11  out_pm10              39102 non-null  float64
 12  out_pm25              39102 non-null  float64
 13  out_so2               39102 non-null  float64
 14  out_temp              3

In [6]:
# Check Frequency
freq = db_client.get_freq(bk_name, ms_name)
print(freq)

0 days 00:02:00


In [7]:
origin_data = origin_data[feature_list]

## Data Refine

In [8]:
# fill in the missing time index
refine_param = {'removeDuplication': {'flag': True} , 'staticFrequency': {'flag': True, 'frequency': "1H"}}
data_refine = DataPreprocessing().get_refinedData(origin_data, refine_param)

In [9]:
data_refine.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2601 entries, 2022-01-11 15:00:00+00:00 to 2022-04-29 23:00:00+00:00
Freq: H
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  1340 non-null   float64
 1   out_coai              1340 non-null   float64
 2   out_h2s               1340 non-null   float64
 3   out_humi              1340 non-null   float64
 4   out_nh3               1340 non-null   float64
 5   out_pm10              1340 non-null   float64
 6   out_temp              1340 non-null   float64
 7   out_ultraviolet_rays  1340 non-null   float64
dtypes: float64(8)
memory usage: 182.9 KB


In [10]:
data_refine.head()

,out_blacksphere_temp,out_coai,out_h2s,out_humi,out_nh3,out_pm10,out_temp,out_ultraviolet_rays
time,,,,,,,,
2022-01-11 15:00:00+00:00,-4.68,90.2,0.0,33.0,0.0,38.2,-2.26,1.3
2022-01-11 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-11 17:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-11 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-11 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
day_start = data_refine.index[0] + timedelta(days=1) - timedelta(hours=data_refine.index[0].hour, 
                                                                 minutes=data_refine.index[0].minute, 
                                                                 seconds=data_refine.index[0].second)

In [12]:
data_refine = data_refine[day_start:]

In [13]:
data_refine

,out_blacksphere_temp,out_coai,out_h2s,out_humi,out_nh3,out_pm10,out_temp,out_ultraviolet_rays
time,,,,,,,,
2022-01-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-04-29 19:00:00+00:00,11.736667,100.0,0.0,68.166667,0.0,0.0,13.233333,1.300000
2022-04-29 20:00:00+00:00,11.253333,100.0,0.0,67.300000,0.0,0.0,12.790000,1.300000
2022-04-29 21:00:00+00:00,11.160000,100.0,0.0,73.433333,0.0,0.0,12.253333,1.300000


## Data Nan -> Mean

In [14]:
method = "mean"
max_limit = {}
parameter = {}

data_refine_nan = BasicImputation(data_refine, method, max_limit, parameter).simpleMethod()

In [15]:
data_refine_nan.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2592 entries, 2022-01-12 00:00:00+00:00 to 2022-04-29 23:00:00+00:00
Freq: H
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  2592 non-null   float64
 1   out_coai              2592 non-null   float64
 2   out_h2s               2592 non-null   float64
 3   out_humi              2592 non-null   float64
 4   out_nh3               2592 non-null   float64
 5   out_pm10              2592 non-null   float64
 6   out_temp              2592 non-null   float64
 7   out_ultraviolet_rays  2592 non-null   float64
dtypes: float64(8)
memory usage: 182.2 KB


## Data Scale

In [16]:
# set parameter
scaler_root_dir ='./scaler/'

clean_train_data_param = 'NoClean'

data_name_x = "AdaRNN_data_x"
feature_list_x= list(data_refine_nan.columns)
x_scaler_file_path = './scaler/AdaRNN_data_x/NoClean/minmax/4457f465de3d0e17dd9f547db5b454b7/scaler.pkl'

split_ratio = 0.8
scaler_param='scale'
scale_method='minmax'

In [17]:
scaler_root_path_x = os.path.join(scaler_root_dir, data_name_x, clean_train_data_param)
test_x, scaler_x = ml_scaler.get_scaled_test_data(data_refine_nan[feature_list_x], x_scaler_file_path, scaler_param)
#test_y, scaler_y = ml_scaler.get_scaled_test_data(data_y[target], y_scaler_file_path, scaler_param)# No Scale

In [18]:
test_x.tail()

,out_blacksphere_temp,out_coai,out_h2s,out_humi,out_nh3,out_pm10,out_temp,out_ultraviolet_rays
time,,,,,,,,
2022-04-29 19:00:00+00:00,0.625239,1.0,0.999969,0.637983,0.999999,0.0,0.584362,0.995873
2022-04-29 20:00:00+00:00,0.614789,1.0,0.999969,0.628127,0.999999,0.0,0.576293,0.995873
2022-04-29 21:00:00+00:00,0.612771,1.0,0.999969,0.697877,0.999999,0.0,0.566525,0.995873
2022-04-29 22:00:00+00:00,0.603402,1.0,0.999969,0.669067,0.999999,0.0,0.560701,0.995873
2022-04-29 23:00:00+00:00,0.605059,1.0,0.999969,0.752085,0.999999,0.0,0.553238,0.995881


## get Data y

In [19]:
LSTMD = LSTMData()
test_x_arr, test_y_arr = LSTMD.transform_Xy_arr(test_x, get_y_data_transformParameter, "NoClean")

1. Original Data Lenagh: 2591
2. Removed Data Length: 0
3. Clean Data Leangth: 2568


In [20]:
test_x_arr.shape, test_y_arr.shape

((2568, 24, 8), (2568, 1))

In [21]:
test_y_arr = test_y_arr.reshape(test_y_arr.shape[0],)

# Data Loader

In [22]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'x_features': 8, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [23]:
test_loader = adarnn.create_testloader(test_x_arr, test_y_arr, shuffle)

In [24]:
list(test_loader)[0][0].shape

torch.Size([64, 24, 8])

# Testing

In [25]:
## model load
adarnn.load_model(model_file_path)

In [26]:
## test
test_loss, test_loss_l1, test_loss_r, pred_list = adarnn.test(test_loader)

Test: 100%|████████████████████████████████████| 41/41 [00:00<00:00, 125.68it/s]

tensor([0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341], device='cuda:0')
tensor([0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
        0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341, 0.0341,
      

In [27]:
test_loss, test_loss_l1, test_loss_r

(0.0006369371087824153, 0.020651765167713165, tensor(0.0004, device='cuda:0'))